In [52]:
# imports
import torch
import torch.nn.functional as F
import torch.nn as nn
import pandas as pd

import torch.optim as optim
import time
from opacus import PrivacyEngine
from vantage6.tools.util import info, warn
from torchvision import transforms
import argparse
from torchvision import datasets, transforms

In [53]:
# simple model 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
#         self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
#         self.conv2_drop = nn.Dropout2d()
#         self.fc1 = nn.Linear(320, 50)
#         self.fc2 = nn.Linear(50, 10)
#
#     def forward(self, x):
#         x = F.relu(F.max_pool2d(self.conv1(x), 2))
#         x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
#         x = x.view(-1, 320)
#         x = F.relu(self.fc1(x))
#         x = F.dropout(x, training=self.training)
#         x = self.fc2(x)
#         return F.log_softmax(x)

In [ ]:
# to check the params (tensors)

# model = Net()

# for parameter in model.parameters():
#     print(parameter)

In [54]:
# initialises training

def RPC_initialize_training(data, gamma, learning_rate, local_dp):
    """
    Initializes the model, optimizer and scheduler and shares the parameters
    with all the workers in the group.

    This should be sent from server to all nodes.

    Args:
        data: contains the local data from the node
        gamma: Learning rate step gamma (default: 0.7)
        learning_rate: The learning rate for training.
        cuda: Should we use CUDA?
        local_dp: bool whether to apply local_dp or not.

    Returns:
        Returns the device, model, optimizer and scheduler.
    """
    
    # Determine the device to train on
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    # print("\033[0;{};49m Rank {} is training on {}".format(device))

    # Initialize model and send parameters of server to all workers
    model = Net().to(device)

    # intializing optimizer and scheduler
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.5)

    # adding DP if true
    if local_dp == True:
        privacy_engine = PrivacyEngine(model, batch_size=64,
                sample_size=60000, alphas=range(2,32), noise_multiplier=1.3,
                max_grad_norm=1.0,)
        privacy_engine.attach(optimizer)

    # returns device, model, optimizer which will be needed in train and test
    return device, model, optimizer    

In [61]:
# basic training of the model

# Question: train gets model, device, optimizer from initialize_training, which is specified within train function, 
# why do I need to call it again before executing the function? Because in vantage6 when I sent the tasks I cannot define that but only in the master function


def RPC_train(data, log_interval, local_dp, epoch, delta=1e-5):
    """
    Training the model on all batches.
    Args:
        epoch: The number of the epoch the training is in.
        round: The number of the round the training is in.
        log_interval: The amount of rounds before logging intermediate loss.
        local_dp: Training with local DP?
        delta: The delta value of DP to aim for (default: 1e-5).
    """
    # loading arguments/parameters from first RPC_method
    device, model, optimizer = RPC_initialize_training(data, gamma, learning_rate, local_dp) # is this allowed in vantage6? calling one RPC_method in another?
     
    train_loader = data
    
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader): 
        # Send the data and target to the device (cpu/gpu) the model is at
        data, target = data.to(device), target.to(device)
        # Clear gradient buffers
        optimizer.zero_grad()
        # Run the model on the data
        output = model(data)
        # Calculate the loss
        loss = F.nll_loss(output, target)
        # Calculate the gradients
        loss.backward()
        # Update model
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                 100. * batch_idx / len(train_loader), loss.item()))
    
    # Adding differential privacy or not
    if local_dp == True:
        epsilon, alpha = optimizer.privacy_engine.get_privacy_spent(delta)
#             print("\033[0;{};49m Epsilon {}, best alpha {}".format(epsilon, alpha))


   

In [84]:
# Model Evaluation

def RPC_test(data, model, device):
    """
    Tests the model.

    Args:
        color: The color for the terminal output for this worker.
        model: The model to test.
        device: The device to test the model on.
        test_loader: The local loader for test local. -> no inside function
    """

    test_loader = data
    
    # device, model, optimizer = RPC_train(data, log_interval, epoch)
     

    model.eval()
    
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            # Send the local and target to the device (cpu/gpu) the model is at
            data, target = data.to(device), target.to(device)
            # Run the model on the local
            output = model(data)
            # Calculate the loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            # Check whether prediction was correct
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

    

In [89]:
"""
These are the parameters needed for the function
Data loading and transforming (this will be done beforehand 
and then stored in './local/training.pt' and './testing.pt')
"""

learning_rate=0.01

log_interval=10 

gamma=0.7



# train_data = torch.utils.data.DataLoader(datasets.MNIST('../mnist_data', 
#                                                           download=True,
#                                                           train=True,
#                                                           transform=transforms.Compose([
#                                                               transforms.ToTensor(), # first, convert image to PyTorch tensor
#                                                               transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
#                                                           ])), 
#                                            batch_size=64, 
#                                            shuffle=True)

# test_data = torch.utils.data.DataLoader(datasets.MNIST('../mnist_data', 
#                                                           download=True,
#                                                           train=False,
#                                                           transform=transforms.Compose([
#                                                               transforms.ToTensor(), # first, convert image to PyTorch tensor
#                                                               transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
#                                                           ])), 
#                                            batch_size=1000, 
#                                            shuffle=True)

# torch.save(train_data, "C:\\Users\\simon\\PycharmProjects\\torch-vantage6\\v6-ppsdg-py\\local\\MNIST\\processed\\training.pt")
# torch.save(test_data, "C:\\Users\\simon\\PycharmProjects\\torch-vantage6\\v6-ppsdg-py\\local\\MNIST\\processed\\testing.pt")


train_loader = torch.load("C:\\Users\\simon\\PycharmProjects\\torch-vantage6\\v6-ppsdg-py\\local\\MNIST\\processed\\training.pt")


test_loader = torch.load("C:\\Users\\simon\\PycharmProjects\\torch-vantage6\\v6-ppsdg-py\\local\\MNIST\\processed\\testing.pt")


local_dp = False

epoch = 3

round = 1

In [91]:
log_interval = 10
local_dp = False
epoch = 3
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)


RPC_test(test_loader, model, device)
for epoch in range(1, epoch + 1):
    torch.manual_seed(1)
    RPC_train(train_loader, log_interval, local_dp, epoch)
    RPC_test(test_loader, model, device)


Test set: Average loss: 2.3118, Accuracy: 1067/10000 (11%)



TypeError: RPC_train() missing 2 required positional arguments: 'local_dp' and 'epoch'

In [71]:
def train(data, data2, log_interval, local_dp, epoch, delta=1e-5):
    train_loader = data
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        # Send the data and target to the device (cpu/gpu) the model is at
        data, target = data.to(device), target.to(device)
        # Clear gradient buffers
        optimizer.zero_grad()
        # Run the model on the data
        output = model(data)
        # Calculate the loss
        loss = F.nll_loss(output, target)
        # Calculate the gradients
        loss.backward()
        # Update model
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                 100. * batch_idx / len(train_loader), loss.item()))

    test_loader = data2

    model.eval()

    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            # Send the local and target to the device (cpu/gpu) the model is at
            data, target = data.to(device), target.to(device)
            # Run the model on the local
            output = model(data)
            # Calculate the loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            # Check whether prediction was correct
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [73]:
for epoch in range(1, epoch + 1):
    train(train_loader, test_loader, 10, False, 10)

Train Epoch: 10 [0/60000 (0%)]	Loss: 0.237544
Train Epoch: 10 [640/60000 (1%)]	Loss: 0.673440
Train Epoch: 10 [1280/60000 (2%)]	Loss: 0.284361
Train Epoch: 10 [1920/60000 (3%)]	Loss: 0.385449
Train Epoch: 10 [2560/60000 (4%)]	Loss: 0.166007
Train Epoch: 10 [3200/60000 (5%)]	Loss: 0.259387
Train Epoch: 10 [3840/60000 (6%)]	Loss: 0.197931
Train Epoch: 10 [4480/60000 (7%)]	Loss: 0.271727
Train Epoch: 10 [5120/60000 (9%)]	Loss: 0.291514
Train Epoch: 10 [5760/60000 (10%)]	Loss: 0.265781
Train Epoch: 10 [6400/60000 (11%)]	Loss: 0.245183
Train Epoch: 10 [7040/60000 (12%)]	Loss: 0.229821
Train Epoch: 10 [7680/60000 (13%)]	Loss: 0.306943
Train Epoch: 10 [8320/60000 (14%)]	Loss: 0.287898
Train Epoch: 10 [8960/60000 (15%)]	Loss: 0.245275
Train Epoch: 10 [9600/60000 (16%)]	Loss: 0.231926
Train Epoch: 10 [10240/60000 (17%)]	Loss: 0.195996
Train Epoch: 10 [10880/60000 (18%)]	Loss: 0.454507
Train Epoch: 10 [11520/60000 (19%)]	Loss: 0.159345
Train Epoch: 10 [12160/60000 (20%)]	Loss: 0.301994
Train Epo

# FED AVG

In [ ]:
# FedAvg gathering of parameters 

def RPC_get_parameters(data, model, parameters):
    """
    Get parameters from nodes
    """
    data_size = len(data) // 3 # number of nodes# size of dataset
    
    RPC_train(data, log_interval, local_dp, epoch, round)

    with torch.no_grad():
        for parameters in model.parameters():
            return {"params": parameters}

In [ ]:
# averaging of returned parameters 

def average_parameters(data, model):
    """
    Get parameters from nodes and calculate the average
    :param model: torch model
    :param parameters: parameters of model
    :param weights:
    :return:
    """
    
    parameters = RPC_get_parameters(data, model, parameters) # makes returned parameters from RPC_get_parameters the parameters used in this function

    # TODO: local: since we usually just get the parameters, this well be an entire task, therefore, we might need to train for each individually


    with torch.no_grad():
        for parameters in model.parameters():
            average = sum(x * y for x, y in zip(parameters[i], weights)) / sum(weights)
            parameters.data = average
            i = i + 1
        return {
            "params_averaged": model
        }
    

#     i = 0
#     with torch.no_grad():
#     for param in model.parameters():
#     # The first entry of the provided parameters when using dist.gather
#     # method also contains the value from the server, remove that one
#     minus_server = parameters[i][1:]
#     # Calculate the average by summing and dividing by the number of
#     # workers
#     s = sum(minus_server)
#     average = s/len(minus_server)
#     # Change the parameter of the global model to the average
#     param.data = average
#     i = i + 1

In [ ]:
data = torch.load("C:\\Users\\simon\\PycharmProjects\\torch-vantage6\\v6-ppsdg-py\\local\\MNIST\\processed\\training.pt")
model = Net()
parameters=model.parameters()
learning_rate=0.01
log_interval=10 
gamma=0.7
round = 1
epoch =1

RPC_get_parameters(data, model, parameters)

In [ ]:
average_parameters(data, model)

In [ ]:
# training with those averaged parameters

def RPC_fed_avg(data, local_dp, epoch, delta=1e-5):
    """
    Training and testing the model on the workers concurrently using federated
    averaging, which means calculating the average of the local model
    parameters after a number of (local) epochs each training round.

    In vantage6, this method will be the training of the model with the average parameters (weighted)

    Returns:
        Returns the final model
    """
    # TODO: local: since we usually just get the parameters, this well be an entire task, therefore, we might need to train for each individually
    model = average_parameters(data, model)
    
    for epoch in range(1, epoch + 1):
        # Train the model on the workers again
        RPC_train(data, local_dp, model, device, optimizer, epoch, delta=1e-5)
        # Test the model on the workers
        RPC_test(data, model, device)

    gather_params = model.get_parameters() # or model.parameters()

    RPC_train(model.RPC_average_parameters_weighted(gather_params))

    return model, parameters


    ## OR 

#     parameters = RPC_average_parameters_weighted(data, model, parameters, weights) # then uses those parameters for training



        # # Gather the parameters after the training round on the server
        #     gather_params = coor.gather_parameters(rank, model, group_size + 1, subgroup)
        #
        #     # If the server
        #     if rank == 0:
        #         # Calculate the average of the parameters and adjust global model
        #         coor.average_parameters_weighted(model, gather_params, weights)
        #
        #     # Send the new model parameters to the workers
        #     coor.broadcast_parameters(model, group)


In [ ]:
RPC_fed_avg(data, local_dp, epoch, delta=1e-5)